Performed Token classification for entities, roles, status and method .
Will be running code multiple times by replacing values in input section as below.
* id_label_temp : place the model id_label mapping dict
* label_type='temp_labels' replace the text temp with entity/role/status/method
* entity_cat='temp' replace the text temp with entity/role/status/method

Evaluation results stored at:
* Model stored at project_directory+'/models/final_models/'+'Indp_'+label_type+'_Model_v1'+'.pth'.
* Evaluation dataset predictions are stored at project_directory+'data/'+entity_cat+'_predictions_'+'tst.json'

### **Metrics:**


***Entity***

{'Alcohol': {'precision': 0.7547169811320755, 'recall': 0.7272727272727273, 'f1': 0.7407407407407407, 'number': 55}, 'Drug': {'precision': 0.8214285714285714, 'recall': 0.7666666666666667, 'f1': 0.793103448275862, 'number': 30}, 'Family': {'precision': 0.5208333333333334, 'recall': 0.6756756756756757, 'f1': 0.5882352941176472, 'number': 37}, 'Tobacco': {'precision': 0.8245614035087719, 'recall': 0.8867924528301887, 'f1': 0.8545454545454546, 'number': 53}, 'overall_precision': 0.7258064516129032, 'overall_recall': 0.7714285714285715, 'overall_f1': 0.7479224376731302, 'overall_accuracy': 0.9720361200116516}


***Method:***


Method_metrics:


{'Method': {'precision': 0.5263157894736842, 'recall': 0.6060606060606061, 'f1': 0.5633802816901409, 'number': 33}, 'overall_precision': 0.5263157894736842, 'overall_recall': 0.6060606060606061, 'overall_f1': 0.5633802816901409, 'overall_accuracy': 0.9752403145936499}




***Role:***

Role_Metrics:


{'Amount': {'precision': 0.41935483870967744, 'recall': 0.7090909090909091, 'f1': 0.5270270270270271, 'number': 55}, 'ExposureHistory': {'precision': 0.14814814814814814, 'recall': 0.4, 'f1': 0.21621621621621623, 'number': 10}, 'Frequency': {'precision': 0.5806451612903226, 'recall': 0.6428571428571429, 'f1': 0.6101694915254238, 'number': 28}, 'Location': {'precision': 0.08333333333333333, 'recall': 0.125, 'f1': 0.1, 'number': 16}, 'QuitHistory': {'precision': 0.07692307692307693, 'recall': 0.06666666666666667, 'f1': 0.07142857142857144, 'number': 15}, 'Temporal': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 11}, 'Type': {'precision': 0.457286432160804, 'recall': 0.6842105263157895, 'f1': 0.5481927710843374, 'number': 133}, 'overall_precision': 0.3708133971291866, 'overall_recall': 0.5783582089552238, 'overall_f1': 0.4518950437317784, 'overall_accuracy': 0.8782406058840664}

***Status***

Status_metrics:
validation:

{'Status': {'precision': 0.5842105263157895, 'recall': 0.5721649484536082, 'f1': 0.578125, 'number': 194}, 'overall_precision': 0.5842105263157895, 'overall_recall': 0.5721649484536082, 'overall_f1': 0.578125, 'overall_accuracy': 0.9472764346053015}




In [ ]:
!pip install evaluate

In [ ]:
import pandas as pd
import evaluate
from transformers import BertTokenizerFast, BertModel, AdamW, get_linear_schedule_with_warmup, DataCollatorForTokenClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
import json

In [ ]:
from tqdm import tqdm, trange

In [ ]:
!pip install seqeval

In [ ]:
'''
Hypertuning parameters  (Method):
eps=1e-8
learning_rate=7e-5
weight_decay=0
num_train_epochs=15
patience=3
batch_size=8

Hypertuning parameters  (Status):
eps=1e-8
learning_rate=5e-5
weight_decay=0
num_train_epochs=15
patience=3
save_model_path='/content/drive/MyDrive/PHD_assessment_gmu/models/final_models/'
batch_size=16

eps=1e-8
learning_rate=7e-5
weight_decay=0
num_train_epochs=15
patience=3
save_model_path='/content/drive/MyDrive/PHD_assessment_gmu/models/final_models/'
batch_size=16
'''



"\nHypertuning parameters  (Method):\neps=1e-8\nlearning_rate=7e-5\nweight_decay=0\nnum_train_epochs=15\npatience=3\nbatch_size=8\n\nHypertuning parameters  (Status):\neps=1e-8\nlearning_rate=5e-5\nweight_decay=0\nnum_train_epochs=15\npatience=3\nsave_model_path='/content/drive/MyDrive/PHD_assessment_gmu/models/final_models/'\nbatch_size=16\n\neps=1e-8\nlearning_rate=7e-5\nweight_decay=0\nnum_train_epochs=15\npatience=3\nsave_model_path='/content/drive/MyDrive/PHD_assessment_gmu/models/final_models/'\nbatch_size=16\n"

In [ ]:
metric=evaluate.load("seqeval")
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizerFast.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')
num_freeze_layers=0
id_label_status={0:'O',1:'B-Status',2:'I-Status'}
id_label_method={0:'O',1:'B-Method',2:'I-Method'}
id_label_role={0:'O',1:'B-Type',2:'I-Type',3:'B-Amount',4:'I-Amount',5:'B-Temporal',6:'I-Temporal',7:'B-Frequency',8:'I-Frequency',9:'B-QuitHistory',10:'I-QuitHistory',11:'B-ExposureHistory',12:'I-ExposureHistory',13:'B-Location',14:'I-Location'}
id_label_event={0:'No Relation',1:'Relation'}
label_id_status = {v: k for k, v in id_label_status.items()}
label_id_method = {v: k for k, v in id_label_method.items()}
label_id_role = {v: k for k, v in id_label_role.items()}
label_id_ent = {'B-Alcohol':1,
 'B-Drug':3,
 'B-Family':5,
 'B-Tobacco':7,
 'I-Alcohol':2,
 'I-Drug':4,
 'I-Family':6,
 'I-Tobacco':8,
 'O':0}
id_label_ent = {v: k for k, v in label_id_ent.items()}
label_id_event = {v: k for k, v in id_label_event.items()}


In [ ]:
#input section
id_label_temp=id_label_ent
label_type='entity_labels'
entity_cat='entity'
project_directory='/content/drive/MyDrive/PHD_assessment_gmu/'


eps=1e-8
learning_rate=7e-5
weight_decay=0
num_train_epochs=15
patience=3
save_model_path=project_directory+'models/final_models/'
batch_size=16

In [ ]:
discarded_enities=['EnvironmentalExposure','SexualHistory','InfectiousDiseases','PhysicalActivity','Residence','LivingSituation','MaritalStatus','Occupation']
discarded_roles=['LivingStatus','Other','MedicalCondition','Extent','History']
max_len=512
min_label_size=10

raw_dataset_path=project_directory+'data/'+'SocialHistoryMTSamples.json'
train_dataset_path=project_directory+'data/'+'trainset.json'
test_dataset_path=project_directory+'data/'+'testset.json'
bert_model_name='bert-base-uncased'

In [ ]:

with open(train_dataset_path,'r') as f:
  trainset=json.load(f)
with open(test_dataset_path,'r') as f:
  testset=json.load(f)

In [ ]:

class GenerateLabel:

  @staticmethod
  def generate_enity_labels(entity_list, token_len, token_offsets):
    '''
    Generates entity labels for each token in the sentence. Performs by mapping the entity's offset positions to the tokens offset.

    Inputs:
    entity_list: List of entity dicts, each entity dict has the information such as offset positions, id , category and text.
    token_len: Initializes the labels size
    token_offsets: list of tuples of offset mappings of bert tokens.
    file_name: Name of the file from which the entity list is generated.

    Outputs:
    entity_labels: List of BIO labels for each token.
    '''
    # Initialize a list to store the BIO labels for each token

    entity_labels = [label_id_ent['O']] * token_len
    for entity in entity_list:
        category = entity['entity_category']
        if category not in discarded_enities:
          entity_start_pos = int(entity['entity_strt_pos'])
          entity_end_pos = int(entity['entity_end_pos'])-1

          # Find tokens that correspond to the entity's position
          entity_start_token = None
          entity_end_token = None

          for i, (start_offset, end_offset) in enumerate(token_offsets):
              if entity_start_token is None and start_offset >= entity_start_pos:
                  entity_start_token = i
              if end_offset > entity_end_pos:
                  entity_end_token = i
                  break

          # Assign BIO labels to the tokens
          if entity_start_token is not None:
            entity_labels[entity_start_token] = label_id_ent['B-' + category]
            if entity_end_token is not None:
              entity_labels[entity_start_token + 1:entity_end_token + 1] = [label_id_ent['I-' + category]] * (entity_end_token - entity_start_token)

    return entity_labels

  @staticmethod
  def generate_role_labels(role_list, token_len, token_offsets):
    '''
    Generates role labels for each token in the sentence. Performs by mapping the entity's offset positions to the tokens offset.
    Also handles overlapping entities by seperating status and method from the role and creating seperate labels for it.

    Inputs:
    role_list: List of role dicts, each entity dict has the information such as offset positions, id , category and text.
    token_len: Initializes the labels size
    token_offsets: list of tuples of offset mappings of bert tokens.
    file_name: Name of the file from which the entity list is generated.

    Outputs:
    role_labels: List of BIO labels for each token.
    status_labels: List of BIO labels for each token.
    method_labels: List of BIO labels for each token.
    '''
    # Initialize a list to store the BIO labels for each token
    role_labels = [label_id_role['O']] * token_len
    status_labels = [label_id_status['O']] * token_len
    method_labels = [label_id_method['O']] * token_len

    for role in role_list:
        category = role['entity_category']
        entity_start_pos = int(role['entity_strt_pos'])
        entity_end_pos = int(role['entity_end_pos'])-1
        if category in discarded_roles:
          continue
        # Find tokens that correspond to the entity's position
        entity_start_token = None
        entity_end_token = None

        for i, (start_offset, end_offset) in enumerate(token_offsets):
            if entity_start_token is None and start_offset >= entity_start_pos:
                entity_start_token = i
            if end_offset > entity_end_pos:
                entity_end_token = i
                break

        # Assign BIO labels to the tokens
        if category == 'Status':
          if entity_start_token is not None:
            status_labels[entity_start_token] = label_id_status['B-' + category]
            if entity_end_token is not None:
                status_labels[entity_start_token + 1:entity_end_token + 1] = [label_id_status['I-' + category]] * (entity_end_token - entity_start_token)

        elif category == 'Method':
          if entity_start_token is not None:
            method_labels[entity_start_token] = label_id_method['B-' + category]
            if entity_end_token is not None:
                method_labels[entity_start_token + 1:entity_end_token + 1] = [label_id_method['I-' + category]] * (entity_end_token - entity_start_token)

        else:
          if entity_start_token is not None:
            role_labels[entity_start_token] = label_id_role['B-' + category]
            if entity_end_token is not None:
                role_labels[entity_start_token + 1:entity_end_token + 1] = [label_id_role['I-' + category]] * (entity_end_token - entity_start_token)


    return role_labels, status_labels, method_labels

  @staticmethod
  def generate_relation_labels( token_offsets, data):
    '''
    Generates relation mapping with token indices for each event present in a document.
    Reads events and maps them to their related entities and roles with their token indices.


    Inputs:
    token_offsets: list of tuples of offset mappings of bert tokens.
    data: Data dict which consists of entity_list, role_list and events_list.

    Outputs:
    relation_labels: The relation mapping is a dictionary where key is a tuple of entity token indices and value is a list of tuple of role token indices.

    '''
    entity_token_indices = {}
    role_token_indices = {}

    entity_list = data.get('entity_list', [])
    role_list = data.get('role_list', [])
    events_list = data.get('events_list', [])

    # Create a dictionary to map entity IDs to their token indices
    for entity in entity_list:
        entity_id = entity['entity_id']
        entity_start_pos = int(entity['entity_strt_pos'])
        entity_end_pos = int(entity['entity_end_pos'])-1

        entity_start_token = None
        entity_end_token = None

        for i, (start_offset, end_offset) in enumerate(token_offsets):
            if entity_start_token is None and start_offset >= entity_start_pos:
                entity_start_token = i
            if end_offset > entity_end_pos:
                entity_end_token = i
                break

        if entity_start_token is not None:
            entity_token_indices[entity_id] = (entity_start_token, entity_end_token)

    # Create a dictionary to map event-related role IDs to their token indices
    for event in events_list:
        entity_id = event['entity_id']
        related_roles = event['Related_roles']

        entity_indices = entity_token_indices.get(entity_id, None)

        if entity_indices:
            role_indices = []

            for role_id in related_roles:
                role = next((role for role in role_list if role['role_id'] == role_id), None)

                if role:
                    role_start_pos = int(role['entity_strt_pos'])
                    role_end_pos = int(role['entity_end_pos'])-1

                    role_start_token = None
                    role_end_token = None

                    for i, (start_offset, end_offset) in enumerate(token_offsets):
                        if role_start_token is None and start_offset >= role_start_pos:
                            role_start_token = i
                        if end_offset > role_end_pos:
                            role_end_token = i
                            break

                    if role_start_token is not None:
                        role_indices.append((role_start_token, role_end_token))

            if role_indices:
                role_token_indices[(entity_indices[0],entity_indices[1])] = role_indices

    return role_token_indices

In [ ]:
def collate_fn_entity_role(batch):
  '''
  Padding inputs to the batch size.
  '''
  input_ids = [torch.tensor(x['input_ids']) for x in batch]
  attention_mask = [torch.tensor(x['attention_mask']) for x in batch]
  #token_type_ids = [torch.tensor(x['token_type_ids']) for x in batch]
  tokens=[x['tokens'] for x in batch]
  file_name=[x['file_name'] for x in batch]

  entity_labels = [torch.tensor(x['entity_labels']) for x in batch]
  role_labels = [torch.tensor(x['role_labels']) for x in batch]
  status_labels = [torch.tensor(x['status_labels']) for x in batch]
  method_labels = [torch.tensor(x['method_labels']) for x in batch]
  input_ids = pad_sequence(input_ids, batch_first=True,padding_value=tokenizer.pad_token_id)
  attention_mask = pad_sequence(attention_mask, batch_first=True,padding_value=0)
  #token_type_ids = pad_sequence(token_type_ids, batch_first=True,padding_value=0)
  entity_labels = pad_sequence(entity_labels, batch_first=True,padding_value=-100)
  role_labels = pad_sequence(role_labels, batch_first=True,padding_value=-100)
  status_labels = pad_sequence(status_labels, batch_first=True,padding_value=-100)
  method_labels = pad_sequence(method_labels, batch_first=True,padding_value=-100)
  return {
    'input_ids':input_ids,
    'attention_mask':attention_mask,
    'entity_labels':entity_labels,
    'role_labels':role_labels,
    'status_labels':status_labels,
    'method_labels':method_labels,
    'tokens':tokens,
    'file_name':file_name,

  }


In [ ]:
class ERDataset(Dataset):
  def __init__(self, data, tokenizer, max_len):
    self.data = data
    self.tokenizer = tokenizer
    self.max_len = max_len
  def __len__(self):
    return len(self.data)
  def __getitem__(self, idx):
    item=self.data[idx]
    text = item['text']
    inputs = self.tokenizer(text,max_length=max_len,truncation=True,return_offsets_mapping=True)
    tokens_len=len(inputs['input_ids'])
    offset_mapping_list=inputs['offset_mapping']
    entity_labels=GenerateLabel.generate_enity_labels(item['entity_list'],tokens_len,offset_mapping_list)
    role_labels, status_labels, method_labels=GenerateLabel.generate_role_labels(item['role_list'],tokens_len,offset_mapping_list)
    relation_labels=GenerateLabel.generate_relation_labels(offset_mapping_list,item)
    return {
      'input_ids':inputs['input_ids'],
      'attention_mask':inputs['attention_mask'],
      'entity_labels':entity_labels,
      'role_labels':role_labels,
      'status_labels':status_labels,
      'method_labels':method_labels,
      'relation_labels':relation_labels,
      'text':text,
      'file_name':item['file_name'],
      'offset_mapping':offset_mapping_list,
      'tokens':inputs.tokens()
    }

In [ ]:
train_data=ERDataset(trainset,tokenizer, max_len)
test_data=ERDataset(testset,tokenizer, max_len)


In [ ]:
train_dataloader = DataLoader(train_data, batch_size=batch_size, collate_fn=collate_fn_entity_role)
test_dataloader = DataLoader(test_data, batch_size=batch_size,collate_fn=collate_fn_entity_role)

In [ ]:
class EntityBertModel(nn.Module):
  def __init__(self, model_name, num_freeze_layers,num_method_labels, dropout=0.1):
    super(EntityBertModel, self).__init__()
    self.bertmodel = BertModel.from_pretrained(model_name)
    for layer in self.bertmodel.encoder.layer[:num_freeze_layers]:
      for param in layer.parameters():
          param.requires_grad = False
    self.dropout = nn.Dropout(dropout)
    #self.status_classifier = nn.Linear(self.bertmodel.config.hidden_size, num_status_labels)
    self.method_classifier = nn.Linear(self.bertmodel.config.hidden_size, num_method_labels)
    #self.role_classifier = nn.Linear(self.bertmodel.config.hidden_size, num_role_labels)
    #self.entity_classifier = nn.Linear(self.bertmodel.config.hidden_size, num_entity_labels)
  def forward(self, input_ids, attention_mask):
    bert_output = self.bertmodel(input_ids=input_ids, attention_mask=attention_mask)
    sequence_output = self.dropout(bert_output[0])
    #status_logits = self.status_classifier(sequence_output)
    method_logits = self.method_classifier(sequence_output)
    #role_logits = self.role_classifier(sequence_output)
    #entity_logits = self.entity_classifier(sequence_output)

    return method_logits#status_logits, method_logits, role_logits, entity_logits

In [ ]:
model= EntityBertModel(model_name=bert_model_name,num_freeze_layers=num_freeze_layers,num_method_labels=len(id_label_temp))

In [ ]:
no_decay=['bias','LayerNorm.weight']
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    }
]


In [ ]:
optimizer = AdamW(model.parameters(), lr=learning_rate,eps=eps,weight_decay=weight_decay)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader)*num_train_epochs)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
training_args={
    'output_dir':save_model_path,
    'num_train_epochs':num_train_epochs,
    'optimizer':optimizer,
    'scheduler':scheduler,
    'patience':patience,
    'run_name':'Indp_'+label_type+'_Model_v1'
}

In [ ]:
def update_nested_labels(entity_logits,ground_truths):
  indices=torch.where(ground_truths>1000)
  index_pairs = list(zip(indices[0].tolist(), indices[1].tolist()))
  probabilities = torch.softmax(entity_logits, dim=-1)
  predictions = torch.argmax(probabilities, dim=-1)
  for i,j in index_pairs:
    multi_label=str(ground_truths[i,j].item())
    parts_str = multi_label.split('900')
    parts_tensors = [int(part) for part in parts_str if part]
    pred_logit=predictions[i, j].item()
    if pred_logit in parts_tensors:
      ground_truths[i,j]=pred_logit
    else:
      ground_truths[i,j]=parts_tensors[0]
  return ground_truths

In [ ]:
def compute_ner_metric(preds,labels,id_label):
    ground_truths=[[id_label[l] for l in lab if l!=-100] for lab in labels]
    prediction_labels=[[id_label[p] for p,l in zip(predict,lab) if l!=-100] for predict,lab in zip(preds,labels)]
    metric_res=metric.compute(predictions=prediction_labels,references=ground_truths)
    return metric_res


In [ ]:
def evaluate_entity_model(model,val_loader,loss_fn,id_label_temp,id_label_ent,id_label_role,id_label_status,id_label_method,label_type,extract_predictions=False,dt_set=''):
  model.eval()
  with torch.no_grad():
    val_ent_loss=0
    val_status_loss=0
    val_method_loss=0
    val_role_loss=0
    val_average_loss=0
    all_entity_labels=[]
    all_entity_predictions=[]
    all_role_labels=[]
    all_role_predictions=[]
    all_method_labels=[]
    all_method_predictions=[]
    all_status_labels=[]
    all_status_predictions=[]
    all_res=[]
    all_res_method={}
    for step,batch in enumerate(val_loader):
      inputs={'input_ids':batch['input_ids'].to(device),'attention_mask':batch['attention_mask'].to(device)}
      #entity_labels=batch['entity_labels'].to(device)
      #role_labels=batch['role_labels'].to(device)
      method_labels=batch[label_type].to(device)
      #status_labels=batch['status_labels'].to(device)
      method_logits=model(**inputs)
      #method_logits=model(**inputs)
      #updated_entity_labels=update_nested_labels(entity_logits,entity_labels)
      #entity_loss=loss_fn(entity_logits.view(-1,len(id_label_ent)),updated_entity_labels.view(-1))
      #role_loss=loss_fn(role_logits.view(-1,len(id_label_role)),role_labels.view(-1))
      #status_loss=loss_fn(status_logits.view(-1,len(id_label_status)),status_labels.view(-1))
      method_loss=loss_fn(method_logits.view(-1,len(id_label_temp)),method_labels.view(-1))
      val_method_loss += method_loss
      method_probabilities=torch.softmax(method_logits,dim=-1)
      method_predictions=torch.argmax(method_probabilities,dim=-1)
      method_labels=batch[label_type].tolist()
      method_predictions=method_predictions.tolist()
      for idx in range(0,len(batch['file_name'])):
        all_res_method[batch['file_name'][idx]]=list(zip(batch['tokens'][idx],method_labels[idx],method_predictions[idx]))



      #all_res.extend(list(zip(tok_lst,lab_lst,pred_lst)) for tok_lst,lab_lst,pred_lst in zip(batch['tokens'],batch['method_labels'].tolist(),method_predictions.tolist()))
      #all_res.append(list(zip(batch['tokens'],batch['method_labels'].tolist(),method_predictions.tolist())))
      all_method_labels.extend(method_labels)
      all_method_predictions.extend(method_predictions)
      #print(all_method_labels)
      #print(all_method_predictions)
      '''
      avg_loss_step=(entity_loss+role_loss+status_loss+method_loss)/4
      val_ent_loss+=entity_loss
      val_role_loss += role_loss
      val_method_loss += method_loss
      val_status_loss += status_loss
      val_average_loss += avg_loss_step

      entity_probabilities = torch.softmax(entity_logits, dim=-1)
      entity_predictions = torch.argmax(entity_probabilities, dim=-1)
      role_probabilities  = torch.softmax(role_logits,dim=-1)
      role_predictions = torch.argmax(role_probabilities, dim =-1)
      status_probabilities=torch.softmax(status_logits,dim=-1)
      status_predictions=torch.argmax(status_probabilities,dim=-1)

      method_probabilities=torch.softmax(method_logits,dim=-1)
      method_predictions=torch.argmax(method_probabilities,dim=-1)
      all_status_predictions.extend(status_predictions.tolist())
      all_status_labels.extend(batch['status_labels'].tolist())
      all_method_labels.extend(batch['method_labels'].tolist())
      all_method_predictions.extend(method_predictions.tolist())
      all_role_labels.extend(batch['role_labels'].tolist())
      all_role_predictions.extend(role_predictions.tolist())
      all_entity_labels.extend(updated_entity_labels.tolist())
      all_entity_predictions.extend(entity_predictions.tolist())
    entity_metrics=compute_ner_metric(all_entity_predictions,all_entity_labels,id_label_ent)
    role_metrics=compute_ner_metric(all_role_predictions,all_role_labels,id_label_role)
    status_metrics=compute_ner_metric(all_status_predictions,all_status_labels,id_label_status)
    '''
    if extract_predictions:
      with open('/content/drive/MyDrive/PHD_assessment_gmu/data/'+entity_cat+'_predictions_'+dt_set+'.json','w') as f:
        json.dump(all_res_method,f)
    method_metrics=compute_ner_metric(all_method_predictions,all_method_labels,id_label_temp)

  return {'avg_loss':val_method_loss/len(val_loader),label_type+'_evaluation':method_metrics}
  #return {'avg_loss':val_average_loss/len(val_loader),'entity_evaluation':(val_ent_loss/len(val_loader),entity_metrics),'role_evaluation':(val_role_loss/len(val_loader),role_metrics),
  #         'method_evaluation':(val_method_loss/len(val_loader),method_metrics),'status_evaluation':(val_status_loss/len(val_loader),status_metrics)}




In [ ]:
def train_entity_bert_model(model,tr_dataloader,tst_dataloader,label_type,id_label_temp,id_label_ent,id_label_role,id_label_status,id_label_method,bert_model_name,training_args):
  if torch.cuda.is_available():
    model.cuda()
  train_cycles=trange(training_args['num_train_epochs'],desc='Epoch',disable=0)
  loss_fn = nn.CrossEntropyLoss(ignore_index=-100)
  optimizer=training_args['optimizer']
  scheduler=training_args['scheduler']
  patience=training_args['patience']
  tr_ent_loss_lst=[]
  tr_status_loss_lst=[]
  tr_method_loss_lst=[]
  tr_role_loss_lst=[]
  tr_avg_loss_lst=[]
  val_ent_loss_list=[]
  val_status_loss_list=[]
  val_method_loss_list=[]
  val_role_loss_list=[]
  val_avg_loss_list=[]

  early_stopping_count_method=0
  for cycle in train_cycles:
    epoch_cycles=tqdm(tr_dataloader,desc='Iteration',disable=-1)
    model.train()
    tr_ent_loss=0
    tr_status_loss=0
    tr_method_loss=0
    tr_role_loss=0
    average_loss=0
    for step,batch in enumerate(epoch_cycles):
      optimizer.zero_grad()
      inputs={'input_ids':batch['input_ids'].to(device),'attention_mask':batch['attention_mask'].to(device)}
      method_logits=model(**inputs)

      method_labels=batch[label_type].to(device)

      method_loss=loss_fn(method_logits.view(-1,len(id_label_temp)),method_labels.view(-1))
      method_loss.backward()
      optimizer.step()
      scheduler.step()
      tr_method_loss+=method_loss.item()
      '''
      entity_labels=batch['entity_labels'].to(device)
      status_logits, method_logits, role_logits, entity_logits=model(**inputs)
      role_labels=batch['role_labels'].to(device)
      method_labels=batch['method_labels'].to(device)
      status_labels=batch['status_labels'].to(device)
      updated_entity_labels=update_nested_labels(entity_logits,entity_labels)
      entity_loss=loss_fn(entity_logits.view(-1,len(id_label_ent)),updated_entity_labels.view(-1))
      role_loss=loss_fn(role_logits.view(-1,len(id_label_role)),role_labels.view(-1))
      status_loss=loss_fn(status_logits.view(-1,len(id_label_status)),status_labels.view(-1))
      method_loss=loss_fn(method_logits.view(-1,len(id_label_method)),method_labels.view(-1))
      avg_loss_step=(entity_loss+role_loss+status_loss+method_loss)/4

      avg_loss_step.backward()
      optimizer.step()
      scheduler.step()
      tr_ent_loss+=entity_loss.item()
      tr_status_loss+=status_loss.item()
      tr_method_loss+=method_loss.item()
      tr_role_loss+=role_loss.item()
      average_loss+=avg_loss_step.item()
      '''
    tr_method_loss_lst.append(tr_method_loss/len(tr_dataloader))

    if early_stopping_count_method == patience or early_stopping_count_method == patience-1:
      tr_results=evaluate_entity_model(model,tr_dataloader,loss_fn,id_label_temp,id_label_ent,id_label_role,id_label_status,id_label_method,label_type,True,'tr')
      val_results=evaluate_entity_model(model,tst_dataloader,loss_fn,id_label_temp,id_label_ent,id_label_role,id_label_status,id_label_method,label_type,True,'tst')
    else:
      val_results=evaluate_entity_model(model,tst_dataloader,loss_fn,id_label_temp,id_label_ent,id_label_role,id_label_status,id_label_method,label_type)
      tr_results=evaluate_entity_model(model,tr_dataloader,loss_fn,id_label_temp,id_label_ent,id_label_role,id_label_status,id_label_method,label_type)

    val_method_loss_list.append(val_results['avg_loss'])
    print('Epoch: {}  Train {} Loss: {}'.format(cycle,label_type,tr_method_loss_lst[-1]))

    print('Epoch: {}  validation {}  Loss: {}'.format(cycle,label_type,val_method_loss_list[-1]))
    print(label_type+'_metrics: \n')
    print('validation:')
    print(val_results[label_type+'_evaluation'])
    print('\n')
    print('Training:')
    print(tr_results[label_type+'_evaluation'])
    print('\n')

    '''
    tr_ent_loss_lst.append(tr_ent_loss/len(tr_dataloader))
    tr_status_loss_lst.append(tr_status_loss/len(tr_dataloader))
    tr_method_loss_lst.append(tr_method_loss/len(tr_dataloader))
    tr_role_loss_lst.append(tr_role_loss/len(tr_dataloader))
    tr_avg_loss_lst.append(average_loss/len(tr_dataloader))
    print('Epoch: {}  Train Entity Loss: {}'.format(cycle,tr_ent_loss_lst[-1]))
    print('Epoch: {}  Train Status Loss: {}'.format(cycle,tr_status_loss_lst[-1]))
    print('Epoch: {}  Train Method Loss: {}'.format(cycle,tr_method_loss_lst[-1]))
    print('Epoch: {}  Train Role Loss: {}'.format(cycle,tr_role_loss_lst[-1]))
    print('Epoch: {}  Train Average Loss: {}'.format(cycle,tr_avg_loss_lst[-1]))
    val_results=evaluate_entity_model(model,tst_dataloader,loss_fn,id_label_ent,id_label_role,id_label_status,id_label_method)
    val_ent_loss_list.append(val_results['entity_evaluation'][0])
    val_status_loss_list.append(val_results['status_evaluation'][0])
    val_method_loss_list.append(val_results['method_evaluation'][0])
    val_role_loss_list.append(val_results['role_evaluation'][0])
    val_avg_loss_list.append(val_results['avg_loss'])
    print('Epoch: {}  Val Entity Loss: {}'.format(cycle,val_ent_loss_list[-1]))
    print('Epoch: {}  Val Status Loss: {}'.format(cycle,val_status_loss_list[-1]))
    print('Epoch: {}  Val Method Loss: {}'.format(cycle,val_method_loss_list[-1]))
    print('Epoch: {}  Val Role Loss: {}'.format(cycle,val_role_loss_list[-1]))
    print('Epoch: {}  Avg Val Loss: {}'.format(cycle, val_avg_loss_list[-1]))
    print('Epoch: {}  Metrics below: \n')
    tr_results=evaluate_entity_model(model,tr_dataloader,loss_fn,id_label_ent,id_label_role,id_label_status,id_label_method)


    print('Entity_metrics: \n')
    print(val_results['entity_evaluation'][1])
    print('\n')
    print(tr_results['entity_evaluation'][1])
    print('\n')
    print('Role_metrics: \n')
    print(val_results['role_evaluation'][1])
    print('\n')
    print(tr_results['role_evaluation'][1])
    print('\n')
    print('Status_metrics: \n')
    print(val_results['status_evaluation'][1])
    print('\n')
    print(tr_results['status_evaluation'][1])
    print('\n')
    print('Method_metrics: \n')
    print(val_results['method_evaluation'][1])
    print('\n')
    print(tr_results['method_evaluation'][1])
    print('\n')
    '''
    if cycle==0:
      min_method_val_loss=val_method_loss_list[-1]
      early_stopping_count_method=0
    else:
      if val_method_loss_list[-1]<min_method_val_loss:
        min_method_val_loss=val_method_loss_list[-1]
        early_stopping_count_method=0
      else:
        early_stopping_count_method+=1
      if early_stopping_count_method>=patience:
        print('Early stopping counter for {} model : {}'.format(label_type,early_stopping_count_method))
        break

    '''
    if cycle==0:
      min_ent_val_loss=val_ent_loss_list[-1]
      min_status_val_loss=val_status_loss_list[-1]
      min_method_val_loss=val_method_loss_list[-1]
      min_role_val_loss=val_role_loss_list[-1]
      min_val_loss=val_results['avg_loss']
      early_stopping_count_ent=0
      early_stopping_count_status=0
      early_stopping_count_method=0
      early_stopping_count_role=0
    else:
      if val_results['avg_loss']<min_val_loss:
        min_val_loss=val_results['avg_loss']
        early_stopping_count=0
      else:
        early_stopping_count+=1
      if val_ent_loss_list[-1]<min_ent_val_loss:
        min_ent_val_loss=val_ent_loss_list[-1]
        early_stopping_count_ent=0
      else:
        early_stopping_count_ent+=1
      if val_status_loss_list[-1]<min_status_val_loss:
        min_status_val_loss=val_status_loss_list[-1]
        early_stopping_count_status=0
      else:
        early_stopping_count_status+=1
      if val_method_loss_list[-1]<min_method_val_loss:
        min_method_val_loss=val_method_loss_list[-1]
        early_stopping_count_method=0
      else:
        early_stopping_count_method+=1
      if val_role_loss_list[-1]<min_role_val_loss:
        min_role_val_loss=val_role_loss_list[-1]
        early_stopping_count_role=0
      else:
        early_stopping_count_role+=1
      if early_stopping_count>=patience or early_stopping_count_ent>=patience or early_stopping_count_status>=patience or early_stopping_count_method>=patience or early_stopping_count_role>=patience:
        print('Early stopping counter for entity model : {}'.format(early_stopping_count_ent))
        print('Early stopping counter for status model : {}'.format(early_stopping_count_status))
        print('Early stopping counter for method model : {}'.format(early_stopping_count_method))
        print('Early stopping counter for role model : {}'.format(early_stopping_count_role))
        print('Early stopping counter for overall model : {}'.format(early_stopping_count))
        print('Early stopping at epoch: {}'.format(cycle))
        break
    '''
  torch.save(model.state_dict(),training_args['output_dir']+training_args['run_name']+'.pth')
  return model , {'train_loss':{'tr_'+label_type+'_loss_lst':tr_method_loss_lst}
                  ,'val_loss':{'val_'+label_type+'_loss_list':val_method_loss_list}}
  #return model, {'train_loss':{'tr_ent_loss_lst':tr_ent_loss_lst,
  #               'tr_method_loss_lst':tr_method_loss_lst,'tr_status_loss_lst':tr_status_loss_lst,'tr_role_loss_lst':tr_role_loss_lst,'tr_avg_loss_lst':tr_avg_loss_lst},
  #               'val_loss':{'val_ent_loss_list':val_ent_loss_list,'val_status_loss_list':val_status_loss_list,'val_method_loss_list':val_method_loss_list,'val_avg_loss_list':val_avg_loss_list}}







In [ ]:
trained_model,loss=train_entity_bert_model(model,train_dataloader,test_dataloader,label_type,id_label_temp,id_label_ent,id_label_role,id_label_status,id_label_method,bert_model_name,training_args)

Epoch:   7%|▋         | 1/15 [00:03<00:54,  3.89s/it]

Epoch: 0  Train entity_labels Loss: 0.5802371922661277
Epoch: 0  validation entity_labels  Loss: 0.43305274844169617
entity_labels_metrics: 

validation:
{'Alcohol': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 55}, 'Drug': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 30}, 'Family': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 37}, 'Tobacco': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 53}, 'overall_precision': 0.0, 'overall_recall': 0.0, 'overall_f1': 0.0, 'overall_accuracy': 0.9236819108651325}


Training:
{'Alcohol': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 199}, 'Drug': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 124}, 'Family': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 146}, 'Tobacco': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 225}, 'overall_precision': 0.0, 'overall_recall': 0.0, 'overall_f1': 0.0, 'overall_accuracy': 0.9171234557800867}




Epoch:  13%|█▎        | 2/15 [00:07<00:49,  3.84s/it]

Epoch: 1  Train entity_labels Loss: 0.37918831145062165
Epoch: 1  validation entity_labels  Loss: 0.3264598548412323
entity_labels_metrics: 

validation:
{'Alcohol': {'precision': 0.9047619047619048, 'recall': 0.34545454545454546, 'f1': 0.5, 'number': 55}, 'Drug': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 30}, 'Family': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 37}, 'Tobacco': {'precision': 0.25925925925925924, 'recall': 0.39622641509433965, 'f1': 0.31343283582089554, 'number': 53}, 'overall_precision': 0.39215686274509803, 'overall_recall': 0.22857142857142856, 'overall_f1': 0.2888086642599278, 'overall_accuracy': 0.9411593358578503}


Training:
{'Alcohol': {'precision': 0.7183098591549296, 'recall': 0.2562814070351759, 'f1': 0.37777777777777777, 'number': 199}, 'Drug': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 124}, 'Family': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 146}, 'Tobacco': {'precision': 0.29155313351498635, 'recall': 0.47

Epoch:  20%|██        | 3/15 [00:11<00:45,  3.83s/it]

...

Epoch: 12  Train entity_labels Loss: 0.042680647991159386
Epoch: 12  validation entity_labels  Loss: 0.1326807141304016
entity_labels_metrics: 

validation:
{'Alcohol': {'precision': 0.7272727272727273, 'recall': 0.7272727272727273, 'f1': 0.7272727272727273, 'number': 55}, 'Drug': {'precision': 0.8518518518518519, 'recall': 0.7666666666666667, 'f1': 0.8070175438596491, 'number': 30}, 'Family': {'precision': 0.5609756097560976, 'recall': 0.6216216216216216, 'f1': 0.5897435897435899, 'number': 37}, 'Tobacco': {'precision': 0.8392857142857143, 'recall': 0.8867924528301887, 'f1': 0.8623853211009174, 'number': 53}, 'overall_precision': 0.7430167597765364, 'overall_recall': 0.76, 'overall_f1': 0.7514124293785311, 'overall_accuracy': 0.9726187008447422}


Training:
{'Alcohol': {'precision': 0.91, 'recall': 0.914572864321608, 'f1': 0.912280701754386, 'number': 199}, 'Drug': {'precision': 0.8818897637795275, 'recall': 0.9032258064516129, 'f1': 0.8924302788844621, 'number': 124}, 'Family': {'pre

Epoch:  93%|█████████▎| 14/15 [00:53<00:03,  3.81s/it]

Epoch: 13  Train entity_labels Loss: 0.039502661565647405
Epoch: 13  validation entity_labels  Loss: 0.13060250878334045
entity_labels_metrics: 

validation:
{'Alcohol': {'precision': 0.7222222222222222, 'recall': 0.7090909090909091, 'f1': 0.7155963302752293, 'number': 55}, 'Drug': {'precision': 0.8214285714285714, 'recall': 0.7666666666666667, 'f1': 0.793103448275862, 'number': 30}, 'Family': {'precision': 0.5102040816326531, 'recall': 0.6756756756756757, 'f1': 0.5813953488372092, 'number': 37}, 'Tobacco': {'precision': 0.8070175438596491, 'recall': 0.8679245283018868, 'f1': 0.8363636363636363, 'number': 53}, 'overall_precision': 0.7074468085106383, 'overall_recall': 0.76, 'overall_f1': 0.7327823691460055, 'overall_accuracy': 0.9711622487620157}


Training:
{'Alcohol': {'precision': 0.9211822660098522, 'recall': 0.9396984924623115, 'f1': 0.9303482587064676, 'number': 199}, 'Drug': {'precision': 0.890625, 'recall': 0.9193548387096774, 'f1': 0.9047619047619047, 'number': 124}, 'Family':

Epoch: 100%|██████████| 15/15 [00:57<00:00,  3.82s/it]

Epoch: 14  Train entity_labels Loss: 0.03564134804422364
Epoch: 14  validation entity_labels  Loss: 0.13035334646701813
entity_labels_metrics: 

validation:
{'Alcohol': {'precision': 0.7547169811320755, 'recall': 0.7272727272727273, 'f1': 0.7407407407407407, 'number': 55}, 'Drug': {'precision': 0.8214285714285714, 'recall': 0.7666666666666667, 'f1': 0.793103448275862, 'number': 30}, 'Family': {'precision': 0.5208333333333334, 'recall': 0.6756756756756757, 'f1': 0.5882352941176472, 'number': 37}, 'Tobacco': {'precision': 0.8245614035087719, 'recall': 0.8867924528301887, 'f1': 0.8545454545454546, 'number': 53}, 'overall_precision': 0.7258064516129032, 'overall_recall': 0.7714285714285715, 'overall_f1': 0.7479224376731302, 'overall_accuracy': 0.9720361200116516}


Training:
{'Alcohol': {'precision': 0.94, 'recall': 0.9447236180904522, 'f1': 0.9423558897243106, 'number': 199}, 'Drug': {'precision': 0.890625, 'recall': 0.9193548387096774, 'f1': 0.9047619047619047, 'number': 124}, 'Family': 

In [ ]:
with open('/content/drive/MyDrive/PHD_assessment_gmu/data/mth_predictions_tst.json','r') as f:
  dt=json.load(f)

In [ ]:
dt[12]

[['[CLS]', 0, 0],
 ['social', 0, 0],
 ['history', 0, 0],
 [':', 0, 0],
 ['he', 0, 0],
 ['lives', 1, 1],
 ['with', 2, 2],
 ['mom', 2, 2],
 [',', 2, 2],
 ['dad', 2, 2],
 [',', 2, 2],
 ['brother', 2, 2],
 [',', 2, 2],
 ['sister', 2, 2],
 [',', 0, 2],
 ['and', 0, 0],
 ['everybody', 0, 0],
 ['is', 0, 0],
 ['healthy', 0, 0],
 ['.', 0, 0],
 ['they', 0, 0],
 ['live', 0, 0],
 ['in', 0, 0],
 ['easton', 0, 0],
 ['.', 0, 0],
 ['they', 0, 0],
 ['have', 0, 0],
 ['4', 0, 0],
 ['dogs', 0, 0],
 [',', 0, 0],
 ['3', 0, 0],
 ['cats', 0, 0],
 [',', 0, 0],
 ['3', 0, 0],
 ['mule', 0, 0],
 ['##s', 0, 0],
 ['and', 0, 0],
 ['no', 0, 0],
 ['deer', 0, 0],
 ['.', 0, 0],
 ['at', 0, 0],
 ['school', 0, 0],
 [',', 0, 0],
 ['he', 0, 0],
 ['is', 0, 0],
 ['in', 0, 0],
 ['second', 0, 0],
 ['grade', 0, 0],
 ['and', 0, 0],
 ['he', 0, 0],
 ['is', 0, 0],
 ['doing', 0, 0],
 ['pe', 0, 0],
 ['without', 0, 0],
 ['any', 0, 0],
 ['limitation', 0, 0],
 ['.', 0, 0],
 ['[SEP]', 0, 0]]